In [7]:
import os

MODEL = 'llama3.2'

file_path = os.path.join(os.getcwd(), r'src\databases\Bizuario Geral.docx')

In [8]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

model.invoke('Me conte uma piada')


'Claro, aqui vai:\n\nPor que a cobra entrou no bar?\n\nPorque queria se refreshar!\n\nE você? Quer ouvir outra piada?'

In [9]:
from langchain_community.document_loaders import Docx2txtLoader
# Adaptar para ler docx ou converter para pdf?

loader = Docx2txtLoader(file_path)
bizuario_doc = loader.load_and_split()
bizuario_doc

[Document(metadata={'source': 'c:\\Users\\Paulo\\Documents\\repos\\rag-chatbot\\src\\databases\\Bizuario Geral.docx'}, page_content='Bizuário Geral Embraer\n\nVOE / DA1 / GME / PPE\n\nAeronaves Executivas Embraer\n\nPlanejamento Estratégico, Engenharia e Precificação de Materiais da Aviação Executiva.\n\n================================================\n\nVSS / DMU / PLN / CPP\n\nSupport and Services\n\nCommercial Parts Engineering & Planning.\n\n\n\n\n\n\n\n\n\nAeronaves Embraer:\n\nExecutiva: Phenom 100  /  Phenom 300  /  Legacy 450  /  Legacy 500  /  Legacy 600  /  Legacy 650  /  Lineage 1000\n\nComercial: ERJ 145 / ERJ 145 XR / ERJ 140 / ERJ 135 / E 195 / E 190 / E 175 / E 170 / E 195 E2 / E 175 E2\n\nMilitar: KC 390 / Super Tucano / Sistemas ISR / Plataformas Multimissão / Transporte de Autoridades \n\nAgricola: Ipanema >> EMB 200 / EMB 200A / EMB 201 / EMB 201A / EMB 202 / EMB 202A / EMB 203\n\n\n\n110: Bandeirante - Plataforma básica \n\n110 DF: Bandeirante Militar/Defesa & Segu

In [10]:
from langchain.prompts import PromptTemplate

# Prompt (System Message. Define Tasks, Tone, Behaviour and Safety params)
template = '''
Você é um assistente virtual de uma área de materiais chamada Spare Parts Planning.
Sua função será responder à questões genéricas feitas pelos colaboradores da área.
Vou lhe passar um documento com diversas informações relevantes, tais como significado de siglas, telefones úteis, explicação de processos, etc, para que você use como referência para responder ao questionamento do usuário.

Siga todas as regras abaixo:
1/ Você deve buscar se comportar de maneira sempre cordial e solícita para atender aos questionamentos dos usuários.

2/ Algumas linhas do documento fornecido podem conter informações irrelevantes. Preste atenção ao conteúdo útil da mensagem.

3/ Existem informações pessoais dos colaboradores no documento, tais como número de telefone, evite passá-las sob quaisquer circunstâncias.
A única informação que pode ser passada relacionada aos colaboradores é o respectivo login.

4/ Em hipótese alguma envolva-se em discussões de cunho pessoal, sobre tecer opiniões sobre um colaborador ou outro. Caso a pergunta seja neste sentido, recuse-se gentilmente a opinar e ofereça ao usuário ajuda nas questões relevantes.

Aqui está uma pergunta recebida de um usuário.
{question}

Aqui está o documento com as informações relevantes mencionadas.
Esse arquivo servirá de base para que você compreenda nosso contexto de negócio, organização e nossas siglas mais comumente utilizadas.
{bizuario_document}

Escreva a melhor resposta que atende ao questionamento do usuário:
'''

prompt = PromptTemplate.from_template(template)
print(prompt.format(context='Here is some context', question='Here is a question'))

KeyError: 'bizuario_document'

In [ ]:
chain.input_schema.schema()

In [ ]:
chain = prompt | model

chain.invoke(
    {
    'context': 'O nome que me foi dado é Paulo',
    'question': 'Qual o meu nome?'
    }
)

In [ ]:
# In this part, VectorStore will be saved in memory only for concluding goals purpose. When developing the app, a fixed store should
# be implemented. Such as sqlite, postgresql or something like that
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(bizuario_doc, embedding=embeddings)


In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke('O que significa EPEP?')

In [ ]:
# Running
from operator import itemgetter

chain = (
    {
        'context': itemgetter('question') | retriever,
        'question': itemgetter('question')
    }
    | prompt
    | model
)

chain.invoke({'question': 'O que significa EPEP?'})

In [ ]:
questions = [
    'O que significa a sigla EPEP?',
    'O que faz a transação ME22N?',
    'Qual transação posso usar para modificar a Ordem de Cliente (OV/SO)?',
    'O que significa APU?',
    'Quais são os processos P3E?',
    'Qual é o ramal do Diego Sodre?',
    'Qual é a chapa do Diego Sodre?',
    'Qual é o ramal do ambulatório?',
    'Quais são algumas das regras para uma boa convivência com os  colegas?',
    'Por que eu devo evitar fazer críticas em público?'
]

for question in questions:
    print(f'Question: {question}')
    print(f'Answer: {chain.invoke({"question": question})}')
    print()

In [ ]:
# Streaming characters
for s in chain.stream({'question': 'O que significa a sigla SLA?'}):
    print(s, end='', flush=True)

In [ ]:
#pip install docarray
#pydantic==1.10.8
